### Getting Data from PostgreSQL DB

Prior to this we have successfull obtained our dataset from Kaggle and migrated it from a `sqlite` database to a `PostgreSQL` database. Now we can query from it using the `psycopg2` package.

#### Required Packages

In [1]:
import pandas as pd       # Data manipulation
import psycopg2           # Query against psql dbs
import numpy as np        # math and arrays
import matplotlib.pyplot as plt # plotting

#### Create Connection

We stored out data in a `PostgreSQL` database named `soccer`.

In [2]:
conn = psycopg2.connect("dbname=soccer")

---

---

### Set up queries

We will write queries for each of our proposed hypothesis tests.

#### 1) Is there a statistical difference in the odds of winning a game when a team is playing in front of their home crowd?

In [74]:
query_1 = """
    select
        c.name as country_name
        ,l.name as league_name
        ,ht.team_short_name as home_team_short_name
        ,ht.team_long_name as home_team_long_name
        ,at.team_short_name as away_team_short_name
        ,at.team_long_name as away_team_long_name
        ,m.season, m.stage, m.date, m.match_api_id
        ,m.home_team_goal, m.away_team_goal
        ,
    from
        match m
        inner join country c on m.country_id = c.id
        inner join league l on m.league_id = l.id
        inner join team ht on m.home_team_api_id = ht.team_api_id
        inner join team at on m.away_team_api_id = at.team_api_id
    ;
"""

#### 2) Impact of defence aggression on average allowed goals. 

In [75]:
query_2 = """
    /* Home Teams */
    select 
        country_name, league_name, team_short_name 
        ,team_long_name, season, stage, match_date
        ,goals_allowed
        ,team_def_aggr_rating
        ,team_def_aggr_scale
        ,'H' as home_or_away
    from
    (
    select
        c.name as country_name
        ,l.name as league_name
        ,ht.team_short_name 
        ,ht.team_long_name 
        ,m.season, m.stage, m.date as match_date, m.match_api_id
        ,m.away_team_goal as goals_allowed
        ,hta.defenceaggression as team_def_aggr_rating
        ,hta.defenceaggressionclass as team_def_aggr_scale
        ,row_number() over (partition by m.home_team_api_id, m.date
            order by to_date(hta.date, 'yyyy-mm-dd')) as enumer
    from
        match m
        inner join country c on m.country_id = c.id
        inner join league l on m.league_id = l.id
        inner join team ht on m.home_team_api_id = ht.team_api_id
        inner join team_attributes hta on hta.team_api_id = m.home_team_api_id
            and to_date(m.date, 'yyyy-mm-dd') 
                <= to_date(hta.date, 'yyyy-mm-dd') 
    ) ranked
    where 
        enumer = 1
        
    union all
    
    /* Away teams */
    select 
        country_name, league_name, team_short_name 
        ,team_long_name, season, stage, match_date
        ,goals_allowed
        ,team_def_aggr_rating
        ,team_def_aggr_scale
        ,'A' as home_or_away
    from
    (
    select
        c.name as country_name
        ,l.name as league_name
        ,at.team_short_name 
        ,at.team_long_name 
        ,m.season, m.stage, m.date as match_date, m.match_api_id
        ,m.home_team_goal as goals_allowed
        ,ata.defenceaggression as team_def_aggr_rating
        ,ata.defenceaggressionclass as team_def_aggr_scale
        ,row_number() over (partition by m.home_team_api_id, m.date
            order by to_date(ata.date, 'yyyy-mm-dd')) as enumer
    from
        match m
        inner join country c on m.country_id = c.id
        inner join league l on m.league_id = l.id
        inner join team at on m.away_team_api_id = at.team_api_id
        inner join team_attributes ata on ata.team_api_id = m.home_team_api_id
            and to_date(m.date, 'yyyy-mm-dd') 
                <= to_date(ata.date, 'yyyy-mm-dd') 
    ) ranked
    where 
        enumer = 1
;
"""

#### 3) Impact of height on heading accuracy.

In [76]:
query_3= """
    select
          *
    from
        player p
        inner join player_attributes pa on p.player_api_id = pa.player_api_id
    ;
"""

#### 4) Impact of team dribbling on win percentage. 

In [3]:
query_4 = """
    /* Home Teams */
    select 
        country_name, league_name, team_short_name 
        ,team_long_name, season, stage, match_date
        ,home_team_goal
        ,away_team_goal
        ,team_dribble_rating
        ,team_dribble_scale
        ,'H' as home_or_away
    from
    (
    select
        c.name as country_name
        ,l.name as league_name
        ,ht.team_short_name 
        ,ht.team_long_name 
        ,m.season, m.stage, m.date as match_date, m.match_api_id
        ,m.home_team_goal
        ,m.away_team_goal
        ,hta.buildupplaydribbling as team_dribble_rating
        ,hta.buildupplaydribblingclass as team_dribble_scale
        ,row_number() over (partition by m.home_team_api_id, m.date
            order by to_date(hta.date, 'yyyy-mm-dd')) as enumer
    from
        match m
        inner join country c on m.country_id = c.id
        inner join league l on m.league_id = l.id
        inner join team ht on m.home_team_api_id = ht.team_api_id
        inner join team_attributes hta on hta.team_api_id = m.home_team_api_id
            and to_date(m.date, 'yyyy-mm-dd') 
                <= to_date(hta.date, 'yyyy-mm-dd') 
    ) ranked
    where 
        enumer = 1
        
    union all
    
    /* Away teams */
    select 
        country_name, league_name, team_short_name 
        ,team_long_name, season, stage, match_date
        ,home_team_goal
        ,away_team_goal
        ,team_dribble_rating
        ,team_dribble_scale
        ,'A' as home_or_away
    from
    (
    select
        c.name as country_name
        ,l.name as league_name
        ,at.team_short_name 
        ,at.team_long_name 
        ,m.season, m.stage, m.date as match_date, m.match_api_id
        ,m.home_team_goal
        ,m.away_team_goal
        ,ata.buildupplaydribbling as team_dribble_rating
        ,ata.buildupplaydribblingclass as team_dribble_scale
        ,row_number() over (partition by m.home_team_api_id, m.date
            order by to_date(ata.date, 'yyyy-mm-dd')) as enumer
    from
        match m
        inner join country c on m.country_id = c.id
        inner join league l on m.league_id = l.id
        inner join team at on m.away_team_api_id = at.team_api_id
        inner join team_attributes ata on ata.team_api_id = m.home_team_api_id
            and to_date(m.date, 'yyyy-mm-dd') 
                <= to_date(ata.date, 'yyyy-mm-dd') 
    ) ranked
    where 
        enumer = 1
;
"""

---

---

### Run queries, put contents into pandas DataFrame

In [4]:
cur = conn.cursor()
cur.execute(query_4)

In [5]:
soccer = pd.DataFrame(cur.fetchall())
soccer.columns = [desc[0] for desc in cur.description]

In [6]:
soccer.head(10).T

,0,1,2,3,4,5,6,7,8,9
country_name,Poland,Poland,Poland,Poland,Poland,Poland,Poland,Poland,Poland,Poland
league_name,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa,Poland Ekstraklasa
team_short_name,CHO,CHO,CHO,CHO,CHO,CHO,CHO,CHO,CHO,CHO
team_long_name,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów,Ruch Chorzów
season,2008/2009,2008/2009,2008/2009,2008/2009,2008/2009,2008/2009,2008/2009,2008/2009,2008/2009,2008/2009
stage,2,3,5,7,9,12,13,15,16,18
match_date,2008-08-16 00:00:00,2008-08-22 00:00:00,2008-09-12 00:00:00,2008-09-27 00:00:00,2008-10-19 00:00:00,2008-11-08 00:00:00,2008-11-11 00:00:00,2008-11-21 00:00:00,2008-11-29 00:00:00,2009-02-28 00:00:00
home_team_goal,2,2,2,0,2,0,1,0,1,0
away_team_goal,1,1,0,0,0,0,2,1,0,1
team_dribble_rating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

---